In [3]:
spark.conf.set(
  "fs.azure.account.key.waginput.blob.core.windows.net",
  "TBwLoPOLim87APX5grtZzWy8Td9h69F/BJgDxuiQyEP480Cs5zyOa2bUeVVRfUnCALOug3aA2Wb4cj8aIiqGEw==")

In [4]:
val dfstation = spark.read.option("header","true").option("inferSchema","true").csv("wasbs://graphdata@waginput.blob.core.windows.net/station.csv")

dfstation: org.apache.spark.sql.DataFrame = [id: int, name: string ... 5 more fields]

In [5]:
display(dfstation)

In [6]:
val dftrip = spark.read.option("header","true").option("inferSchema","true").csv("wasbs://graphdata@waginput.blob.core.windows.net/trip.csv")

dftrip: org.apache.spark.sql.DataFrame = [id: int, duration: int ... 9 more fields]

In [7]:
display(dftrip)

In [8]:
dfstation.printSchema()
dftrip.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dock_count: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- installation_date: string (nullable = true)

root
 |-- id: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- start_date: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- end_date: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- subscription_type: string (nullable = true)
 |-- zip_code: string (nullable = true)

In [9]:
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions.col

import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions.col

In [10]:
val justStations = dfstation.selectExpr("float(id) as station_id", "name").distinct()

justStations: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [station_id: float, name: string]

In [11]:
val stations = dftrip.select("start_station_id").withColumnRenamed("start_station_id", "station_id").union(dftrip.select("end_station_id").withColumnRenamed("end_station_id", "station_id")).distinct().select(col("station_id").cast("long").alias("value"))

stations: org.apache.spark.sql.DataFrame = [value: bigint]

In [12]:
val stationVertices: RDD[(VertexId, String)] = stations.join(justStations, stations("value") === justStations("station_id")).select(col("station_id").cast("long"), col("name")).rdd.map(row => (row.getLong(0), row.getString(1))) // maintain type information

stationVertices: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, String)] = MapPartitionsRDD[49] at map at <console>:36

In [13]:
val stationEdges:RDD[Edge[Long]] = dftrip.select(col("start_station_id").cast("long"), col("end_station_id").cast("long")).rdd.map(row => Edge(row.getLong(0), row.getLong(1), 1))

stationEdges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Long]] = MapPartitionsRDD[54] at map at <console>:34

In [14]:
val defaultStation = ("Missing Station") 
val stationGraph = Graph(stationVertices, stationEdges, defaultStation)
stationGraph.cache()

defaultStation: String = Missing Station
stationGraph: org.apache.spark.graphx.Graph[String,Long] = org.apache.spark.graphx.impl.GraphImpl@13537857
res7: org.apache.spark.graphx.Graph[String,Long] = org.apache.spark.graphx.impl.GraphImpl@13537857

In [16]:
println("Total Number of Stations: " + stationGraph.numVertices)
println("Total Number of Trips: " + stationGraph.numEdges)
// sanity check
println("Total Number of Trips in Original Data: " + dftrip.count)

Total Number of Stations: 70
Total Number of Trips: 669959
Total Number of Trips in Original Data: 669959

In [ ]:
val ranks = stationGraph.pageRank(0.0001).vertices
ranks.join(stationVertices).sortBy(_._2._1, ascending=false).take(10).foreach(x => println(x._2._2))

Error from http request: OK: 

In [19]:
stationGraph.groupEdges((edge1, edge2) => edge1 + edge2).triplets.sortBy(_.attr, ascending=false).map(triplet => "There were " + triplet.attr.toString + " trips from " + triplet.srcAttr + " to " + triplet.dstAttr + ".").take(10).foreach(println)

There were 1849 trips from San Francisco Caltrain 2 (330 Townsend) to Townsend at 7th.
There were 1809 trips from Townsend at 7th to San Francisco Caltrain 2 (330 Townsend).
There were 1743 trips from Harry Bridges Plaza (Ferry Building) to Embarcadero at Sansome.
There were 1728 trips from Harry Bridges Plaza (Ferry Building) to Embarcadero at Sansome.
There were 1688 trips from Townsend at 7th to San Francisco Caltrain (Townsend at 4th).
There were 1653 trips from Harry Bridges Plaza (Ferry Building) to Embarcadero at Sansome.
There were 1551 trips from San Francisco Caltrain 2 (330 Townsend) to Townsend at 7th.
There were 1475 trips from 2nd at Townsend to Harry Bridges Plaza (Ferry Building).
There were 1439 trips from Townsend at 7th to San Francisco Caltrain (Townsend at 4th).
There were 1424 trips from San Francisco Caltrain 2 (330 Townsend) to Townsend at 7th.

In [20]:
stationGraph.inDegrees.join(stationVertices).sortBy(_._2._1, ascending=false).take(10).foreach(x => println(x._2._2 + " has " + x._2._1 + " in degrees."))

San Francisco Caltrain (Townsend at 4th) has 63179 in degrees.
San Francisco Caltrain 2 (330 Townsend) has 35117 in degrees.
Harry Bridges Plaza (Ferry Building) has 33193 in degrees.
Embarcadero at Sansome has 30796 in degrees.
2nd at Townsend has 28529 in degrees.
Market at Sansome has 28033 in degrees.
Townsend at 7th has 26637 in degrees.
Steuart at Market has 25025 in degrees.
Temporary Transbay Terminal (Howard at Beale) has 23080 in degrees.
Market at 4th has 19915 in degrees.

In [21]:
stationGraph.outDegrees.join(stationVertices).sortBy(_._2._1, ascending=false).take(10).foreach(x => println(x._2._2 + " has " + x._2._1 + " out degrees."))

San Francisco Caltrain (Townsend at 4th) has 49092 out degrees.
San Francisco Caltrain 2 (330 Townsend) has 33742 out degrees.
Harry Bridges Plaza (Ferry Building) has 32934 out degrees.
Embarcadero at Sansome has 27713 out degrees.
Temporary Transbay Terminal (Howard at Beale) has 26089 out degrees.
2nd at Townsend has 25837 out degrees.
Steuart at Market has 24838 out degrees.
Market at Sansome has 24172 out degrees.
Townsend at 7th has 23724 out degrees.
Market at 10th has 20272 out degrees.

In [22]:
stationGraph.inDegrees.join(stationGraph.outDegrees).join(stationVertices).map(x => (x._2._1._1.toDouble/x._2._1._2.toDouble, x._2._2)).sortBy(_._1, ascending=false).take(5).foreach(x => println(x._2 + " has a in/out degree ratio of " + x._1))

Redwood City Medical Center has a in/out degree ratio of 1.4533762057877813
Redwood City Public Library has a in/out degree ratio of 1.300469483568075
San Francisco Caltrain (Townsend at 4th) has a in/out degree ratio of 1.286951030717836
Washington at Kearney has a in/out degree ratio of 1.2548577376821652
MLK Library has a in/out degree ratio of 1.233038348082596

In [23]:
stationGraph.inDegrees.join(stationGraph.inDegrees).join(stationVertices).map(x => (x._2._1._1.toDouble/x._2._1._2.toDouble, x._2._2)).sortBy(_._1).take(5).foreach(x => println(x._2 + " has a in/out degree ratio of " + x._1))

San Jose Diridon Caltrain Station has a in/out degree ratio of 1.0
San Jose Civic Center has a in/out degree ratio of 1.0
Santa Clara at Almaden has a in/out degree ratio of 1.0
Adobe on Almaden has a in/out degree ratio of 1.0
San Pedro Square has a in/out degree ratio of 1.0